In [1]:
using Random, Distributions
using LinearAlgebra
using Gurobi, JuMP
using DataFrames
using CSV

In [2]:
# Create a gurobi model without the annoying academic license message
gurobi_env = Gurobi.Env()
function create_gurobi_model(; TimeLimit=-1, LogFile="logs.txt")
    model = Model(optimizer_with_attributes(() -> Gurobi.Optimizer(gurobi_env)));
    if TimeLimit >= 0
        println("Set Gurobi TimeLimit.")
        set_optimizer_attribute(model, "TimeLimit", TimeLimit)
    end
    set_optimizer_attribute(model, "LogFile", LogFile)
    set_optimizer_attribute(model, "OutputFlag", 0)
    set_optimizer_attribute(model, "NumericFocus", 3)
    return model
end;


--------------------------------------------
--------------------------------------------

Academic license - for non-commercial use only


In [3]:
function extendβ(β_true_p)
    β_true = zeros(2*p)
    for i in 1:2*p
        if i<=p
            β_true[i] = max(β_true_p[i],0)
        else
            β_true[i] = max(-β_true_p[i-p],0)
        end
    end
    return β_true
end

extendβ (generic function with 1 method)

In [10]:
Random.seed!(2021)

# Params
n, p = 100, 10

# Robustness
γ = 10

# Significance
α = 0.05
t_α = 1 - quantile(TDist(n-p), α/2) # Beware: n-p-1 if we add intercept

# Data
X_p = rand(n, p) 
X = hcat(X_p,-X_p)

β_true_p = [rand([0,1])*randn()*10 for i in 1:p]
β_true = extendβ(β_true_p)


σ_noise = 0.001


#y = rand(n)
y = X*β_true + [randn() for i in 1:n] * σ_noise

# Test
X_test_p = rand(n, p)
X_test = hcat(X_test_p,-X_test_p)
y_test = X_test*β_true + [randn() for i in 1:n] * σ_noise

# Variance estimator
M_p = X_p'X_p
M = X'X
M_inv_p = M_p^-1
σ_tilde_p = sqrt((y'*(I - X_p*M_inv_p*X_p')*y)/(n-p))

diag_M_inv_p = [max(x,0) for x in diag(M_inv_p)]
σ_X_p = σ_tilde_p * sqrt.(diag_M_inv_p)
σ_X = [σ_X_p; σ_X_p]

# # Sparsity
k = 5

5

In [5]:
function g(s,X,y,γ)
    # Compute matrices
    Z = Diagonal(s)
    # Compute D
    D = (I/γ + Z*M)^(-1)
    
    # Compute norm
    function compute_DZ_square_norm(in_norm)
        return in_norm' * D*Z * in_norm
    end
    
    # Compute max
    model = create_gurobi_model()
    @variable(model, λ[1:2*p] >= 0)
    
    obj_1 = 0.5*y'y
    obj_2 = t_α*λ'*(s.*σ_X)
    obj_3 = - 0.5 * compute_DZ_square_norm(X'y + λ)
    @objective(model, Max, obj_1 + obj_2 + obj_3)

    optimize!(model)

    # Compute β

    sparsity_indexes = findall(x->x>0, s)
    X_s = X[:, sparsity_indexes]
    λ_s = value.(λ)[sparsity_indexes]
    β_s = (I/γ + X_s'X_s)^(-1)*(X_s'y + λ_s)
    
    β_pred = zeros(2*p)
    β_pred[sparsity_indexes] = β_s
    return β_pred, value.(λ), objective_value(model)
    
end

g (generic function with 1 method)

In [6]:
function grad_g(s,X,y,λ,γ)
    grad = zeros(2*p)
    Z = Diagonal(s)
    # Compute D
    D = (I/γ + Z*M)^(-1)
    # Compute norm

    function compute_DED_square_norm(E,in_norm)
        return in_norm' * D*E*D' * in_norm
    end
    for i in 1:2*p
        E_ii = Diagonal([(j == i)*1 for j in 1:2*p])
        grad[i] = t_α*λ'E_ii*σ_X - 0.5*compute_DED_square_norm(E_ii, X'y+ λ)
    end
    return grad
end

grad_g (generic function with 1 method)

In [7]:
function dual(X,y,γ,k)
    miop = create_gurobi_model()
    @variable(miop, s[1:2*p], Bin)
    @variable(miop, t)
    @constraint(miop, sum(s) <= k)
    @constraint(miop, [i=1:p], s[i]+s[p+i]<=1)
    #initial linear constraint of cutting planes
    s0 = zeros(2*p)
    s0[1:k] .= 1
    (β_0,λ,p0) = g(s0,X,y,γ)

    ∇s0 = grad_g(s0,X,y,λ,γ)
    
    @constraint(miop, t >= p0 + dot(∇s0, s - s0))
    @objective(miop, Min, t)
    
    function outer_approximation(cb_data)
        s_val = [callback_value(cb_data, s[i]) for i=1:2*p]
        (β_pred,λ,obj) = g(s_val,X,y,γ)
        ∇s = grad_g(s_val,X,y,λ,γ)
        offset = sum(∇s .* s_val)
        con = @build_constraint(t >= obj + ∇s'*s - offset)
        MOI.submit(miop, MOI.LazyConstraint(cb_data), con)
    end
    
    MOI.set(miop, MOI.LazyConstraintCallback(), outer_approximation)
    optimize!(miop)
    
    s_opt = JuMP.value.(s)
    (β_pred,λ,obj) = g(s_opt,X,y,γ)
    return s_opt, β_pred, objective_value(miop)
end

dual (generic function with 1 method)

In [11]:
s_opt, β_pred, obj = dual(X,y,γ,k)

([-0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, -0.0, 1.0, -0.0, 0.0, -0.0, -0.0, -0.0, 1.0, -0.0, 1.0, 1.0, -0.0, 1.0], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 2.50903, 0.0, 0.0, 0.0, 0.0, 0.0, 16.1047, 0.0, 26.7265, 4.49984, 0.0, 1.68358], 50.493063728611844)

In [12]:
hcat(β_true, β_pred)

20×2 Array{Float64,2}:
  0.0       0.0    
  0.0       0.0    
  0.0       0.0    
  0.0       0.0    
  0.0       0.0    
  0.0       0.0    
  0.0       0.0    
  0.0       0.0    
  2.64798   2.50903
  0.0       0.0    
  0.0       0.0    
  0.0       0.0    
  0.0       0.0    
  0.0       0.0    
 16.2361   16.1047 
  0.0       0.0    
 26.9009   26.7265 
  4.46339   4.49984
  0.0       0.0    
  1.59877   1.68358

In [239]:
function get_primal(γ)
    model = create_gurobi_model()

    big_M = 1000
    big_M_sig = 1000

    @variable(model, β[i=1:p])
    @variable(model, s[i=1:p], Bin)
    @variable(model, b[i=1:p], Bin)

    @constraint(model, sum(s) <= k)
    @constraint(model, [i=1:p], β[i] <= big_M*s[i])
    @constraint(model, [i=1:p], β[i] >= -big_M*s[i])

    @constraint(model, [i=1:p], β[i]/σ_X_p[i] + big_M_sig*b[i] >= t_α*s[i])
    @constraint(model, [i=1:p], -β[i]/σ_X_p[i] + big_M_sig*(1-b[i]) >= t_α*s[i])

    @objective(model, Min, 0.5*sum((y[i] - X_p[i,:]'β)^2 for i=1:p) + 1/(2*γ) * sum(β[i]^2 for i=1:p))
#    @objective(model, Min, 0.5*sum((y[i] - X[i,:]'β)^2 for i=1:p))

    optimize!(model)
    
    cat = DataFrame(b = value.(b), β=value.(β), s=value.(s))
    
    return objective_value(model), value.(β), cat
end

get_primal (generic function with 1 method)

In [240]:
function get_insample_R2(y_pred, y_true)
    TSE = sum((y_pred[i]-y_true[i])^2 for i=1:p)
    baseline_E = sum((sum(y_true)/length(y_true)-y_true[i])^2 for i=1:p)
    return 1 - TSE/baseline_E
end

get_insample_R2 (generic function with 1 method)

In [241]:
function get_OR2(y_pred, y_true, y_train)
    TSE = sum((y_pred[i]-y_true[i])^2 for i=1:p)
    baseline_E = sum((sum(y_train)/length(y_train)-y_true[i])^2 for i=1:p)
    return 1 - TSE/baseline_E
end

get_OR2 (generic function with 1 method)

In [242]:
obj, get_primal(γ)

(-17.969264130473903, [0.0, 0.0335249, 0.0, 0.0332135, 1.11327e-12, 0.0334871, 0.0332761, 0.0, 0.0, 0.0331722], 10×3 DataFrame
│ Row │ b       │ β           │ s           │
│     │ Float64 │ Float64     │ Float64     │
├─────┼─────────┼─────────────┼─────────────┤
│ 1   │ -0.0    │ 0.0         │ -0.0        │
│ 2   │ -0.0    │ 0.0335249   │ 1.0         │
│ 3   │ -0.0    │ 0.0         │ -0.0        │
│ 4   │ -0.0    │ 0.0332135   │ 1.0         │
│ 5   │ -0.0    │ 1.11327e-12 │ 3.35002e-11 │
│ 6   │ -0.0    │ 0.0334871   │ 1.0         │
│ 7   │ -0.0    │ 0.0332761   │ 1.0         │
│ 8   │ -0.0    │ 0.0         │ -0.0        │
│ 9   │ -0.0    │ 0.0         │ -0.0        │
│ 10  │ -0.0    │ 0.0331722   │ 1.0         │)